In [27]:
import pandas as pd
weather=pd.read_csv('/kaggle/input/indian-weather-repository-daily-snapshot/IndianWeatherRepository.csv')
weather.head()

,country,location_name,region,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,India,Ashoknagar,Madhya Pradesh,24.57,77.72,Asia/Kolkata,1693286100,2023-08-29 10:45,27.5,81.5,...,12.6,18.5,1,2,05:59 AM,06:41 PM,05:42 PM,03:38 AM,Waxing Gibbous,93
1,India,Raisen,Madhya Pradesh,23.33,77.80,Asia/Kolkata,1693286100,2023-08-29 10:45,27.5,81.5,...,10.7,14.2,1,1,06:00 AM,06:40 PM,05:39 PM,03:41 AM,Waxing Gibbous,93
2,India,Chhindwara,Madhya Pradesh,22.07,78.93,Asia/Kolkata,1693286100,2023-08-29 10:45,26.3,79.3,...,16.8,20.7,2,2,05:56 AM,06:34 PM,05:32 PM,03:39 AM,Waxing Gibbous,93
3,India,Betul,Madhya Pradesh,21.86,77.93,Asia/Kolkata,1693286100,2023-08-29 10:45,25.6,78.1,...,4.9,6.6,1,1,06:00 AM,06:38 PM,05:36 PM,03:43 AM,Waxing Gibbous,93
4,India,Hoshangabad,Madhya Pradesh,22.75,77.72,Asia/Kolkata,1693286100,2023-08-29 10:45,27.2,81.0,...,11.4,14.8,1,1,06:01 AM,06:39 PM,05:38 PM,03:42 AM,Waxing Gibbous,93


In [28]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Load stock data
stock_data = pd.read_csv("/kaggle/input/nse-data/bhavdata.csv")

# Load weather data
weather_data = pd.read_csv("/kaggle/input/indian-weather-repository-daily-snapshot/IndianWeatherRepository.csv")

# Round timestamps in both datasets
stock_data['TIMESTAMP'] = pd.to_datetime(stock_data['TIMESTAMP'], format='%d-%b-%Y')
weather_data['last_updated'] = pd.to_datetime(weather_data['last_updated']).dt.round('D')


In [29]:
weather_data.columns

Index(['country', 'location_name', 'region', 'latitude', 'longitude',
       'timezone', 'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')

In [30]:
aggregated_weather_data = weather_data.groupby('last_updated').agg({
    'temperature_celsius': 'mean',
    'wind_kph': 'mean',
    'wind_degree': 'mean',
   
    'pressure_in': 'mean',
     'precip_in': 'sum',
    'humidity': 'mean',
    'cloud': 'mean',
    'feels_like_celsius': 'mean',
    'visibility_km': 'mean',
    'uv_index': 'mean'
}).reset_index()


In [31]:
aggregated_weather_data.dtypes

last_updated           datetime64[ns]
temperature_celsius           float64
wind_kph                      float64
wind_degree                   float64
pressure_in                   float64
precip_in                     float64
humidity                      float64
cloud                         float64
feels_like_celsius            float64
visibility_km                 float64
uv_index                      float64
dtype: object

In [32]:
label_encoder = LabelEncoder()
stock_data['SERIES'] = label_encoder.fit_transform(stock_data['SERIES'])
stock_data.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
0,1018GS2026,31,128.00,128.00,128.00,128.00,128.00,131.50,578,73984.0,2023-12-11,4,IN0020010081
1,182D040424,77,97.69,97.69,97.69,97.69,97.69,97.59,100,9769.0,2023-12-11,1,IN002023Y284
2,182D060624,77,96.47,96.56,96.47,96.56,96.56,96.52,2200,212274.0,2023-12-11,8,IN002023Y375
3,182D070324,77,98.16,98.16,98.16,98.16,98.16,98.02,400,39264.0,2023-12-11,2,IN002023Y243
4,182D090524,77,97.04,97.04,97.04,97.04,97.04,97.00,200,19408.0,2023-12-11,2,IN002023Y334


In [33]:
merged_data = pd.merge(stock_data, aggregated_weather_data, left_on=['TIMESTAMP'], right_on=['last_updated'], how='inner')


In [34]:
merged_data.columns

Index(['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'LAST', 'PREVCLOSE',
       'TOTTRDQTY', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES', 'ISIN',
       'last_updated', 'temperature_celsius', 'wind_kph', 'wind_degree',
       'pressure_in', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'visibility_km', 'uv_index'],
      dtype='object')

In [55]:

# Merge datasets on rounded timestamps and location information

# Delete rows with timestamps that don't map
merged_data.dropna(subset=['TIMESTAMP'], inplace=True)

# Feature selection
features = ['OPEN','LAST', 'PREVCLOSE','SERIES', 'temperature_celsius']


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(merged_data[features], merged_data['CLOSE'], test_size=0.2, random_state=42)


# Train CatBoost model with weather data



In [61]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

model = Sequential()
model.add(GRU(units=64, activation='relu', input_shape=(X_train.shape[1], 1),return_sequences=True))
model.add(GRU(units=32, activation='relu',return_sequences=True))
model.add(GRU(units=16, activation='relu',return_sequences=True))
model.add(GRU(units=8, activation='relu',return_sequences=True))
model.add(GRU(units=4, activation='relu',return_sequences=True))
model.add(GRU(units=2, activation='relu'))
model.add(Dense(units=1, activation='relu'))  # Output layer with one neuron for regression task

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model

callbacks = [
       tf.keras.callbacks.ModelCheckpoint(
    filepath='models/model_{epoch:02d}', monitor='val_loss',mode = 'min' , verbose=1, save_best_only=True, save_weights_only=False)]
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),callbacks=callbacks)

Epoch 1/50
6322/6323 [============================>.] - ETA: 0s - loss: 536903.5000
Epoch 1: val_loss improved from inf to 1220.16260, saving model to models/model_01
6323/6323 [==============================] - 150s 22ms/step - loss: 536829.1875 - val_loss: 1220.1626
Epoch 2/50
6323/6323 [==============================] - ETA: 0s - loss: 72131.4922
Epoch 2: val_loss did not improve from 1220.16260
6323/6323 [==============================] - 128s 20ms/step - loss: 72131.4922 - val_loss: 1533.0933
Epoch 3/50
6323/6323 [==============================] - ETA: 0s - loss: 79183.7422
Epoch 3: val_loss did not improve from 1220.16260
6323/6323 [==============================] - 126s 20ms/step - loss: 79183.7422 - val_loss: 27222.9082
Epoch 4/50
6321/6323 [============================>.] - ETA: 0s - loss: 31434.7910
Epoch 4: val_loss did not improve from 1220.16260
6323/6323 [==============================] - 127s 20ms/step - loss: 31425.8613 - val_loss: 1517.4814
Epoch 5/50
6322/6323 [======

In [65]:
# Assuming predictions_with_weather is the scaled predictions obtained from your model
model.load_weights('/kaggle/working/models/model_41')
model.predict(X_test)

1581/1581 [==============================] - 9s 6ms/step


array([[ 360.1666 ],
       [ 181.46384],
       [  16.9167 ],
       ...,
       [1101.9252 ],
       [ 842.9146 ],
       [ 144.36612]], dtype=float32)

In [63]:
y_test

198235     360.00
53195      172.10
235338      17.66
184208     164.35
175950      31.65
           ...   
175711      27.90
76645      103.75
164399    1110.00
11791      846.70
19527      143.50
Name: CLOSE, Length: 50584, dtype: float64

### We can see that predicted value are close to the real value
#### We have used only one additional Feature `Mean Temperature` and can see the effect
#### We plan to create a Multi Modal Model for this task as mentioned in the ppt